In [ ]:
print("Hello Virus!")

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import datetime

In [ ]:
# REFERENCE CODE

In [ ]:
# Store CSVs in dataframes
dataset_1 = ("Data/2019_nCoV_data.csv")

nCoV_df = pd.read_csv(dataset_1)
nCoV_df.head()

In [ ]:
# pandas drop columns using list of column names
nCoV_df = nCoV_df.drop(['Sno', 'Province/State', 'Last Update'], axis=1)
nCoV_df.head()

In [ ]:
# Either transform the data, or create new dataframes with only selected columns

# Transform:
#new_premise_file_df = premise_file_df[["Premises Name", "County ID Code"]].copy()
#new_premise_file_df.columns = ["premise_name", "county_id"]
#new_premise_file_df.head()


# Create new df with select columns:
#new_customer_data_df = customer_data_df[['id', 'first_name', 'last_name']].copy()
#new_customer_data_df.head()

In [ ]:
# For BERNT:
# How to rename columns 
#new_premise_file_df.columns = ["premise_name", "county_id"]

In [ ]:
# Connect to database

rds_connection_string = "postgres:postgres@localhost:5432/<name_of_database>_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
# Check for tables

engine.table_names()

In [ ]:
# Use pandas to load CSV into SQL database

#new_customer_data_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)

In [ ]:
# Use PGAdmin to verify that data loaded.

# Alternatively, try:
#pd.read_sql_query('select * from customer_name', con=engine).head()

In [ ]:
# REFERENCE CODE ENDS

In [ ]:
# KAAN Code Starts

In [ ]:
dataset_2 = "Data\sars_2003_complete_dataset_clean.csv"
sars_df = pd.read_csv(dataset_2, parse_dates = ["Date"])
sars_df.head()

In [ ]:
#renaming the columns to ensure consistency when the data is loaded into a database. We found these metrics to be most relevant for this dataset. 

sars_df.columns = ['date', 'country', 'confirmed_cases', 'deaths', 'recoveries']
sars_df.head()

In [ ]:

#there will be multiple different virus types that will be loaded into the database so adding the virus type as a new column

sars_df['virus_name'] = "SARS"
sars_df.head()

In [ ]:
#because there are multiple date formats in each data, we decided to seperate it into year/month/date to keep it consistent across the board 

sars_df['year'] = pd.DatetimeIndex(sars_df['date']).year
sars_df['month'] = pd.DatetimeIndex(sars_df['date']).month
sars_df['day'] = pd.DatetimeIndex(sars_df['date']).day
sars_df.head()

In [ ]:
sars_df['day_number'] = (sars_df['date'] - min(sars_df['date'])).dt.days + 1
sars_df.head()

In [ ]:
#check for naming conventions on countries and make sure they're consistent before uploading to the database. here for instance, china is listed multiple times under different names 
sars_df["country"].unique()

In [ ]:
sars_df = sars_df.replace(to_replace ="Taiwan, China", 
                 value ="Taiwan") 

sars_df = sars_df.replace(to_replace ="Hong Kong SAR, China", 
                 value ="Hong Kong") 

sars_df = sars_df.replace(to_replace ="Macao SAR, China", 
                 value ="Macao") 

sars_df = sars_df.replace(to_replace ="Russian Federation", 
                 value ="Russia") 

sars_df = sars_df.replace(to_replace ="Viet Nam", 
                 value ="Vietnam") 

sars_df = sars_df.replace(to_replace ="Republic of Ireland", 
                 value ="Ireland") 

sars_df = sars_df.replace(to_replace ="Republic of Korea", 
                 value ="South Korea") 


sars_df

In [ ]:
# KAAN Code Ends

In [ ]:
# BERNT Code Starts

In [ ]:
# Load in df's
dataset_1 = ("Data/covid_19_data.csv")

nCoV_df = pd.read_csv(dataset_1, parse_dates = ["ObservationDate"])
nCoV_df.head()

In [ ]:
# pandas drop columns using list of column names
nCoV_df = nCoV_df.drop(['SNo', 'Province/State', 'Last Update'], axis=1)
nCoV_df.head()

In [ ]:
# rename columns 
nCoV_df.columns = ["date", "country", "confirmed_cases", "deaths", "recoveries"]
nCoV_df

In [ ]:
# Here we are counting what day the incident occurred - based on the first entry recorded by country.
# We discussed this, as we were very interested in trying to figure out how to bin the data.
# We realized that that was getting into data analysis, and decided to leave more granular data 
# available for our database. That way any future analyst could decide for themselves what they'd 
# like to do with it. 

nCoV_df['day_number'] = (nCoV_df['date'] - min(nCoV_df['date'])).dt.days + 1
#nCoV_df.iloc[2000]
nCoV_df

In [ ]:
nCoV_df['virus_name'] = 'Covid19'
nCoV_df.tail()

In [ ]:
nCoV_df['year'] = pd.DatetimeIndex(nCoV_df['date']).year
nCoV_df['month'] = pd.DatetimeIndex(nCoV_df['date']).month
nCoV_df['day'] = pd.DatetimeIndex(nCoV_df['date']).day

nCoV_df

In [ ]:
nCoV_df["country"].unique()

In [ ]:
nCoV_df = nCoV_df.replace(to_replace ="Mainland China", 
                 value ="China") 
nCoV_df = nCoV_df.replace(to_replace ="Macau", 
                 value ="Macao") 
nCoV_df = nCoV_df.replace(to_replace ="UK", 
                 value ="United Kingdom")
nCoV_df = nCoV_df.replace(to_replace = "US", 
                value = "United States")

#nCoV_df.head()

nCoV_df["country"].sort_values(ascending=True).unique()

In [ ]:
nCoV_df

In [ ]:
# Group the data by Country/column type
test = nCoV_df.groupby(['Country', 'Day #', 'Virus_Name'])['Confirmed_Cases', 'Deaths', 'Recoveries'].sum()
#test.loc['Italy']
test

In [ ]:
# Connect to database

rds_connection_string = "postgres:postgres@localhost:5432/virus_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
# Check for tables

engine.table_names()

In [ ]:
# Use pandas to load CSV into SQL database

sars_df.to_sql(name='sars', con=engine, if_exists='append', index=False)
nCoV_df.to_sql(name='covid_19', con=engine, if_exists='append', index=False)

In [ ]:
nCoV_df.head()

In [ ]:
sars_df.head()

In [ ]:
# Use PGAdmin to verify that data loaded.

# Alternatively, try:
#pd.read_sql_query('select * from customer_name', con=engine).head()

In [ ]:
# Extraneous Code

# Extract numeric values for just Month and Year
#nCoV_df['Year_Month'] = pd.to_datetime(nCoV_df['Date']).dt.to_period('M')
#nCoV_df.tail()

#test['Minimum_Month'] = min(nCoV_df["Year_Month"])
#test


#nCoV_df['Bins'] = pd.cut(x=nCoV_df['Day #'], bins=[0, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70],
                         #labels=['Week 1', 'Week 2', 'Week 3', 'Week 4', 'Week 5', 'Week 6',
                                 #'Week 7', 'Week 8', 'Week 9', 'Week 10'])
#nCoV_df

#df_ages['age_by_decade'] = pd.cut(x=df_ages['age'], bins=[20, 29, 39, 49], labels=['20s', '30s', '40s'])

In [ ]:
# BERNT Code Ends